In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from keras_tqdm import TQDMNotebookCallback

from data import im_generator_cifar
from unet import unet

Using TensorFlow backend.


In [2]:
# print models as seen by tf
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [3]:
model = unet(input_size=(32, 32, 3))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/zaccharie/workspace/understanding-unets/unet.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input=inputs, output=conv6)


In [ ]:
batch_size = 32
im_gen_train = im_generator_cifar(mode='training', validation_split=0.1, batch_size=batch_size)
im_gen_val = im_generator_cifar(mode='validation', validation_split=0.1, batch_size=batch_size)
im_gen_test = im_generator_cifar(mode='testing', batch_size=batch_size)

In [ ]:
history = model.fit_generator(
    im_gen_train, 
    steps_per_epoch=int(0.9*5*1e4 / 32), 
    epochs=50,
    validation_data=im_gen_val,
    validation_steps=int(0.1*5*1e4 / 32),
#     validation_freq=2,
    verbose=0,
#     use_multiprocessing=True,
    callbacks=[TQDMNotebookCallback()],
)

Instructions for updating:
Use tf.cast instead.


In [ ]:
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

In [ ]:
plt.figure(figsize=(9, 5))
for key, val in history.history.items():
    plt.plot(np.log(val), label=key)
plt.legend()

In [ ]:
batch_test_noisy, batch_test_gt = next(im_gen_test)

In [ ]:
batch_test_pred = model.predict(batch_test_noisy)

In [ ]:
fig, axs = plt.subplots(32, 3, figsize=(14, 32*5))
for example_id in range(32):
    axs[example_id, 0].imshow(batch_test_gt[example_id])
    axs[example_id, 0].set_title("original image")
    axs[example_id, 1].imshow(batch_test_noisy[example_id])
    axs[example_id, 1].set_title("noisy image")
    axs[example_id, 2].imshow(batch_test_pred[example_id])
    axs[example_id, 2].set_title("denoised image")